In [2]:
!pip install PyMuPDF

     |████████████████████████████████| 8.8 MB 5.2 MB/s 


In [3]:
import glob
import re
import os
import fitz
import numpy as np
import pandas as pd
import math
import json
import pprint

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
import spacy

import gensim.corpora as corpora
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import collections

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "



In [4]:
# --------------------------- Read a pdf into a large string of text ---------------------------
def read_pdf(file_path):
    pymupdf_text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            pymupdf_text += page.getText()
    return pymupdf_text


# --------------------------- Read a report and breaks it up into individual sentences ---------------------------
def convert_pdf_into_sentences(text):
    # Remove unnecessary spaces and line breaks
    text = re.sub(r'\x0c\x0c|\x0c', "", str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('\n', ' ', str(text))
    text = ' '.join(text.split())
    text = " " + text + "  "
    text = text.replace("\n", " ")
    if "”" in text: text = text.replace(".”", "”.")
    if "\"" in text: text = text.replace(".\"", "\".")
    if "!" in text: text = text.replace("!\"", "\"!")
    if "?" in text: text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]

    # Filter for sentences with more than 100 characters
    sentences = [s.strip() for s in sentences if len(s) > 100]
    return sentences


# --------------------------- Retrieve the report name from the pdf ---------------------------
def reportName(path):
    name = path.split('/')[-1]
    company = name.split('.')[0]
    return company

In [5]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
pdf_path = '/content/drive/My Drive/BT4222/ESG_Report.pdf'

In [7]:
# Read our database of ESG reports
esg_reports = [pdf_path]
esg_corpus = {}
for report in esg_reports:
    esg_corpus[reportName(report)] = convert_pdf_into_sentences(read_pdf(report))
    print(f"Reading Report '{reportName(report)}'")

Reading Report 'ESG_Report'


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [9]:
# Read the key words from our json file
f = open('/content/drive/My Drive/BT4222/keywords.json')
keywordBank = json.load(f)
f.close()


In [10]:
keywordBank

{'ESG phrases': ['best-in-class',
  'carbon footprint',
  'carbon pricing',
  'clean technology',
  'engagement',
  'environmental factors',
  'esg integration',
  'ethical investing',
  'exclusions',
  'negative screening',
  'governance factors',
  'green bond',
  'greenwashing',
  'human rights',
  'impact investments',
  'modern slavery',
  'PRI',
  'proxy voting',
  'renewable energy',
  'screening',
  'social factors',
  'SRI',
  'stewardship',
  'thematic investing',
  'SDG',
  'values-based investing',
  'voting rights',
  'biodiversity',
  'carbon capture and storage',
  'circular economy',
  'climate action tracker',
  'climate clocks',
  'climate funds',
  'climate transition benchmarks',
  'greenhouse gas emissions',
  'net zero carbon pledge and initiative',
  'paris agreement',
  'paris-aligned benchmarks',
  'PFAS',
  'scope 1',
  'scope 2',
  'scope 3',
  'sdg funds',
  'sin stocks',
  'smart esg scores',
  'social sustainability',
  'stewardship code',
  'stranded asse

In [12]:
def read_pdf(file_path):
    pymupdf_text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            pymupdf_text += page.getText()
    return pymupdf_text

In [46]:
pdf_path = '/content/drive/My Drive/BT4222/ESG_Report4.pdf'

In [14]:
# function takes in a report and breaks it up into individual sentences
def convert_pdf_into_sentences(text):

    # remove unnecessary spaces and line breaks
    text = re.sub(r'\x0c\x0c|\x0c', "", str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('\n', ' ', str(text))
    text = ' '.join(text.split())
    text = " " + text + "  "
    text = text.replace("\n", " ")
    if "”" in text: text = text.replace(".”", "”.")
    if "\"" in text: text = text.replace(".\"", "\".")
    if "!" in text: text = text.replace("!\"", "\"!")
    if "?" in text: text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]

    #filter for sentences with more than 100 characters
    sentences = [s.strip() for s in sentences if len(s) > 100]
    return sentences

In [47]:
doc = read_pdf(pdf_path)
sentences = convert_pdf_into_sentences(doc)
sentences[:20]

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


['Annual report 2021 Annual report 2021 All amounts in million EUR (unless stated otherwise) 2014 2015 2016 2017 2018 2019 2020 Mensch und Maschine at a glance Revenue 140.',
 '3% * Comparison 2019/18 distorted by first time application of IFRS16 2021 Management report 2 Course of business 2021 20 Statement of income 25 Balance sheet 26 Statement of cash flows 27 Development of shareholders’ equity 27 Notes 28 Segment reporting 28 Auditor’s report 65 Administrative Board report 70 Addresses 72 Content Markus Pech CFO Christoph Aschenbrenner COO Adi Drotleff Chairman Introduction Dear reader, After the turbulent year 2020 - beginning with a very strong Q1, followed by three quarters with Corona retarding effects - M+M in 2021 continued its record chasing, clearly outperforming the pre-crisis year 2019 by +28% at EBIT level.',
 '7 Million, as well as 126 Cents net profit/share and 218 Cents cash flows per share, we achieved new records on all levels despite the ongoing Corona crisis.',
 

In [18]:
def keySentences(corpus, subFeatureKeywords):
    sentencesCaptured = []
    for word in subFeatureKeywords:
        sentencesCaptured.extend([sentence for sentence in corpus if word in sentence])
    return sentencesCaptured

In [22]:
keywordBank['Environment']['Opportunities in Clean Tech']

['renewable',
 'energy efficiency',
 'smart',
 'efficient',
 'efficiency',
 'cleantech',
 'green technologies',
 'energy-saving',
 'energy',
 'green',
 'technology']

In [23]:
keySentences(sentences, keywordBank['Environment']['Opportunities in Clean Tech'])

['When we talk about our investment in research and development in furtherance of our smart data technology, we do so with an eye to our responsibilities to our investors, customers, and other stakeholders.',
 'SMART DATA Using our “smart data” to deliver smarter analytics, we multiply the continuing innovation in artificial intelligence and machine learning and new technologies.',
 'Demonstrating Product Leadership with Sustainability by Design 14 EMPLOYEES GOVERNANCE PLANET CUSTOMERS COMMUNITIES NETSCOUT 2021 Environmental, Social, and Governance Report 14 Reduced disk usage via “smart data” Through our powerful Adaptive Session Trace (AST) technology, NETSCOUT appliances store only the packet information of interest to users and reduces unnecessary transportation of that data through the network.',
 'This is highly efficient because it avoids deploying two separate physical devices or software instances in the same location.',
 'Hyperscale cloud environments allow workloads to run h

In [28]:
keywordBank.keys()

dict_keys(['Environment', 'Social', 'Governance', 'ESG phrases'])

In [48]:
dict_ESG4 = {}
for i in ['Environment', 'Social', 'Governance']:
  for j in keywordBank[i].keys():
    a = keySentences(sentences, keywordBank[i][j])
    print('\n ===', j, '===\n', str(len(a)))
    dict_ESG4[j] = len(a)
    for k in a:
      print(k)


 === Opportunities in Clean Tech ===
 14
Not only does this improve quality and efficiency of excavation and earth moving work significantly, but also relieves the increasingly urgent shortage of skilled workforce in the construction industry, as it firstly improves the attractiveness of the job in the excavator cab especially for younger and technology-savvy people, and it secondly saves the previously obligatory ‘second man with the yardstick’ on site.
11 Management report Digitization in practice: Building Information Modeling (BIM) Project: High efficiency gain through BIM in Interior Design Customer: BMS2 GmbH, Munich, Germany All disciplines of a building project working together in 3D, collaborating, and sharing information via a common database, that’s BIM - also known as 5D, as time and quantity/cost dimensions are usually additionally attributed to the model.
Similar solutions based on MapEdit are available, for example to check the optimal positions for electric car chargin

In [38]:
print('ESG Report 1') #ESG
dict_ESG

ESG Report 1


{'Carbon Emissions': 7,
 'Electronic Waste': 17,
 'Governance': 44,
 'Human Capital Development': 15,
 'Labor Management': 22,
 'Opportunities in Clean Tech': 30,
 'Privacy and Data Security': 81,
 'Toxic Emissions & Waste': 5,
 'Water Stress': 1}

In [41]:
print('ESG Report 2') #AR with ESG Report inside
dict_ESG2

ESG Report 2


{'Carbon Emissions': 11,
 'Electronic Waste': 48,
 'Governance': 761,
 'Human Capital Development': 213,
 'Labor Management': 366,
 'Opportunities in Clean Tech': 82,
 'Privacy and Data Security': 189,
 'Toxic Emissions & Waste': 0,
 'Water Stress': 9}

In [45]:
print('ESG Report 3') #ESG Report
dict_ESG3

ESG Report 3


{'Carbon Emissions': 30,
 'Electronic Waste': 23,
 'Governance': 78,
 'Human Capital Development': 7,
 'Labor Management': 83,
 'Opportunities in Clean Tech': 41,
 'Privacy and Data Security': 62,
 'Toxic Emissions & Waste': 10,
 'Water Stress': 22}

In [49]:
print('ESG Report 4') #AR with ESG Report inside
dict_ESG4

ESG Report 4


{'Carbon Emissions': 8,
 'Electronic Waste': 15,
 'Governance': 166,
 'Human Capital Development': 38,
 'Labor Management': 92,
 'Opportunities in Clean Tech': 14,
 'Privacy and Data Security': 45,
 'Toxic Emissions & Waste': 1,
 'Water Stress': 0}

In [16]:
# function for gensim preprocessing
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True)) 

In [17]:
# function for creating stopwords (english stopwords, context specific words, company names)
def create_stopwords(companyname):
    stop_words = companyname + stopwords.words('english') 
    stop_words.extend(['accounting', 'active', 'income', 'adventure', 'allocation', 'shares', 'amortization', 'amplitude', 'annuity', 'appreciation', 'arbitrage', 'ask', 'asset', 'asset approach', 'aval'])
    stop_words.extend(['plc', 'group', 'target', 'track', 'capital', 'holding', 'report', 'annualreport', 'esg', 'bank', 'report', 'annualreport', 'long', 'make', 'fy'])
    return stop_words
    
# function for removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

### Extract out features such as:

    - PDF Document Name
    - Sentence Count (i.e Length of PDF)
    - ESG Sentences + Percentage of Occurence
    - Env Sentence + Percentage of Occurence
    - Social Sentence + Percentage of Occurence
    - Gov Sentence + Percentage of Occurence
    - Env Sentiment Score 
    - Social Sentiment Score 
    - Gov Sentiment Score 
    - Averaged ESG Sentiment Score

In [11]:
# --------------------------- Sentiment Analysis ---------------------------
# This function calculates the sentiment score for the various sentences using VADER
# Sentence:
#   - The sentence to be inputted to the function, which will return the respective sentiment score
#   - If there are > 1 sentence, the average will be computed and returned
def averagedCompoundSentimentScore(sentences):
    sid = SentimentIntensityAnalyzer()
    score = 0
    for sentence in sentences:
        sentiment = sid.polarity_scores(sentence)
        score += sentiment['compound']
    try:
        return score / len(sentences)
    except ZeroDivisionError:
        return score





# --------------------------- Sentence Extraction ---------------------------
# This function extracts out the keywords from the given corpus
# corpus: 
#   - This refers to a document (i.e one company)
# subFeatureKeywords:
#   - A list containing all the keywords which we would like to identify from our sentence bank
def keySentences(corpus, subFeatureKeywords):
    sentencesCaptured = []
    for word in subFeatureKeywords:
        sentencesCaptured.extend([sentence for sentence in corpus if word in sentence])
    return sentencesCaptured





# --------------------------- Print all sentences (Debugging purposes only) ---------------------------
def printAllSentences(corpus, pillar, keywordBank):
    for subFeature, kewords in keywordBank[pillar].items():
        print('\n\n\n')
        print(f"======= Printing Sentences from: '{subFeature}' =======")
        sentences = keySentences(corpus, kewords)
        for sentence in sentences:
            print(sentence)
            print('\n\n')
        




# --------------------------- Subpillar Feature Statistics ---------------------------
# 4 options for pillar: 
#   - 'Environment'
#   - 'Social'
#   - 'Governance'
#   - 'ESG phrases'
# corpus: 
#   - A specific company report, and NOT the whole collection of reports from all companies!
# keywordBank: 
#   - All the keywords from the subpillar
def subPillar_featureStats(corpus, pillar, keywordBank):
    data = {}

    # Calculate the sentences, frequency of sentence occurence, sentiment score etc
    def summaryStatistics(corpus, subFeatureKeywords):
        temp = {
            # "Sentences": None,
            "NumOfSentences": None,
            "FrequencyOfOccurence": None,
            "SentimentScore": None
        }
        # temp['Sentences'] = subpillar_sentences(corpus, keywordBank[pillar])
        sentences = keySentences(corpus, subFeatureKeywords)
        temp['NumOfSentences'] = len(sentences)
        temp['FrequencyOfOccurence'] = round(len(sentences) / len(corpus), 5)
        temp['SentimentScore'] = averagedCompoundSentimentScore(sentences)
        return temp

    for subFeature, subFeatureKeywords in keywordBank[pillar].items():
        data[subFeature] = summaryStatistics(corpus, subFeatureKeywords)
    
    return data





# --------------------------- Complete Feature Statistics ---------------------------
# This combines all the data across the 3 pillars into a dictionary 
# esg_bank:
#   - Complete set of data processed from reading in all the companies
#   - Structure of esg_bank:
#       - Dictionary where
#           - key: company name
#           - value: [sentence1, sentence2, ..., sentenceN]
# companyName:
#   - The company we wish to explore
# keywordBank:
#   - Complete set of data from the keywords.json file
def featureStats(esg_bank, companyName, keywordBank):
    company = {
        companyName: []
    }
    for pillar in [*keywordBank][:-1]:
        temp = {}
        temp[pillar] = subPillar_featureStats(esg_bank[companyName], pillar, keywordBank)
        company[companyName].append(temp)
    return company
    


def processByLength(esg_bank, keywordBank, numberOfReports):
    print('\n\n === Generating feature statistic data from all companies === \n\n')
    companies = []

    def flatten_data(dictionary_data):
        new_data = {
            "Company Name": list(dictionary_data.keys())[0],
        }
        avgscore = []
        for subData in dictionary_data.values():
            for i in range(0, 3):
                for pillar, pillarValues in subData[i].items():
                    title = f"avg {pillar} Sentiment Score"
                    try:   
                        avg = round(sum([data['SentimentScore'] for title, data in pillarValues.items()]) / len(pillarValues), 5)
                        new_data[title] = avg
                        avgscore.append(avg)
                    except ZeroDivisionError:
                        new_data[title] = 0
                        avgscore.append(0)
        new_data['avg ESG Sentiment'] = round(sum(avgscore) / len(avgscore), 5)
        return new_data

    counter = 0
    for company, data in esg_bank.items():
        if counter == numberOfReports:
            break
        else:
            print(f"Processing data from --- {company}")
            company_data = featureStats(esg_bank, company, keywordBank)
            companies.append(flatten_data(company_data))
            counter += 1

    return pd.DataFrame(companies)


In [ ]:
# --------------------------- Weights for each subpillar ---------------------------
# import from weights.json 
# The weights have been calibrated to sum up to 100%

f = open('weights.json')
weights = json.load(f)
pprint.pprint(weights)
f.close()

{'Environment': {'Carbon Emissions': 5.7,
                 'Electronic Waste': 0.6,
                 'Opportunities in Clean Tech': 11.4,
                 'Toxic Emissions & Waste': 0.3,
                 'Water Stress': 1.9},
 'Governance': {'Governance': 41.2},
 'Social': {'Human Capital Development': 20.8,
            'Labor Management': 5.8,
            'Privacy and Data Security': 12.3}}


In [ ]:
processByLength(esg_corpus, keywordBank, 20)



 === Generating feature statistic data from all companies === 


Processing data from --- Mobvista, Inc
Processing data from --- NICE Ltd
Processing data from --- Jack Henry _ Associates Inc 2020
Processing data from --- Iflytek Co
Processing data from --- Integrated Research Ltd
Processing data from --- Meituan 2020
Processing data from --- Logicom Public Ltd
Processing data from --- PagSeguro Digital Ltd
Processing data from --- ManTech International Corp
Processing data from --- Hyundai Autoever Corp
Processing data from --- NTT DATA Corp
Processing data from --- network-2021-prelims_090322
Processing data from --- NetEnt AB 2019
Processing data from --- home24 SE 2018
Processing data from --- Medallia, Inc
Processing data from --- Salesforce
Processing data from --- SII - AF_RA_Annuel_2020_2021_EN_0
Processing data from --- Izertis SA 2021
Processing data from --- Marvelous Inc
Processing data from --- NetEase, Inc


,Company Name,avg Environment Sentiment Score,avg Social Sentiment Score,avg Governance Sentiment Score,avg ESG Sentiment
0,"Mobvista, Inc",0.06563,0.39254,0.40645,0.28821
1,NICE Ltd,0.23740,0.46952,0.52577,0.41090
2,Jack Henry _ Associates Inc 2020,0.29226,0.52483,0.51878,0.44529
3,Iflytek Co,-0.00183,0.52517,0.42056,0.31463
4,Integrated Research Ltd,0.37847,0.35297,0.35777,0.36307
5,Meituan 2020,0.17271,0.43422,0.43090,0.34594
6,Logicom Public Ltd,0.21378,0.36782,0.32991,0.30384
7,PagSeguro Digital Ltd,0.24411,0.38277,0.41574,0.34754
8,ManTech International Corp,0.33431,0.43215,0.41813,0.39486
9,Hyundai Autoever Corp,0.28896,0.36376,0.42687,0.35986


In [ ]:
pprint.pprint(featureStats(esg_corpus, 'NTT DATA Corp', keywordBank))

{'NTT DATA Corp': [{'Environment': {'Carbon Emissions': {'FrequencyOfOccurence': 0.00621,
                                                         'NumOfSentences': 2,
                                                         'SentimentScore': 0.9915},
                                    'Electronic Waste': {'FrequencyOfOccurence': 0.00621,
                                                         'NumOfSentences': 2,
                                                         'SentimentScore': -0.6249},
                                    'Opportunities in Clean Tech': {'FrequencyOfOccurence': 0.17081,
                                                                    'NumOfSentences': 55,
                                                                    'SentimentScore': 0.3519509090909091},
                                    'Toxic Emissions & Waste': {'FrequencyOfOccurence': 0.0,
                                                                'NumOfSentences': 0,
                   

In [ ]:
subPillar_featureStats(esg_corpus['IAC_InteractiveCorp 2020'], 'Environment', keywordBank)

{'Opportunities in Clean Tech': {'NumOfSentences': 9,
  'FrequencyOfOccurence': 0.05625,
  'SentimentScore': 0.8066444444444445},
 'Carbon Emissions': {'NumOfSentences': 20,
  'FrequencyOfOccurence': 0.125,
  'SentimentScore': 0.45719999999999994},
 'Water Stress': {'NumOfSentences': 1,
  'FrequencyOfOccurence': 0.00625,
  'SentimentScore': 0.9468},
 'Electronic Waste': {'NumOfSentences': 2,
  'FrequencyOfOccurence': 0.0125,
  'SentimentScore': 0.30679999999999996},
 'Toxic Emissions & Waste': {'NumOfSentences': 0,
  'FrequencyOfOccurence': 0.0,
  'SentimentScore': 0}}

In [ ]:
subPillar_featureStats(esg_corpus['IAC_InteractiveCorp 2020'], 'Social', keywordBank)

{'Human Capital Development': {'NumOfSentences': 8,
  'FrequencyOfOccurence': 0.05,
  'SentimentScore': 0.8763625},
 'Privacy and Data Security': {'NumOfSentences': 50,
  'FrequencyOfOccurence': 0.3125,
  'SentimentScore': 0.48931800000000003},
 'Labor Management': {'NumOfSentences': 25,
  'FrequencyOfOccurence': 0.15625,
  'SentimentScore': 0.37038400000000005}}

In [ ]:
# df = featureStatistic(esg_corpus, esg_keywords, environment_features, social_features, governance_features)
# df